In [1]:
!pip install transformers datasets trl peft bitsandbytes accelerate torch sentencepiece huggingface_hub -U
!pip install --upgrade trl transformers accelerate
!pip install --upgrade deepspeed==0.14.4

# ATTENTION: Be sure to restart the notebook after installing all packages!

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 104.7 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 71.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.1/411.1 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 40.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 48.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 5.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 10.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 103.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.

# Log into Hugging Face
Ensure that you have access to the `meta-llama/Llama-3.2-3B-Instruct` gated respository on Hugging Face (you can visit [https://huggingface.co/meta-llama/Llama-3.2-3B-Instruct] to request access). Access to the base model is required before running inference on the fine-tuned model.

In [2]:
from huggingface_hub import interpreter_login
interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



# Prompt
Edit the code below to change the user prompt. Run the cell below to get the fine-tuned Llama 3.2 3B model's response.

In [ ]:
prompt_text = "Investigating the effects of manipulation on the gale shapley algorithm."

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
import warnings
from IPython.display import display, Markdown

# Suppress warnings if needed
warnings.filterwarnings("ignore")

# --- Configuration ---
base_model_id = "meta-llama/Llama-3.2-3B-Instruct"
# *** IMPORTANT: Update this path to where your LoRA adapter weights are saved ***
adapter_path = "./results_llama3_finetuned"
# Set device (use "cuda" if GPU is available, otherwise "cpu")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Optional: Quantization config for loading base model (if needed for memory)
# Adjust based on your inference hardware and how you trained/saved.
# If you merge first, you might load the base without quantization here.
use_quantization = False # Set to True if you want to load base model in 4-bit
compute_dtype = torch.bfloat16 # Or torch.float16

bnb_config = None
if use_quantization:
    print("Loading base model with 4-bit quantization...")
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False, # Typically False for inference
    )
else:
    print(f"Loading base model with dtype: {compute_dtype}")


# --- Load Base Model and Tokenizer ---
print(f"Loading base model: {base_model_id}")
model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    quantization_config=bnb_config,
    torch_dtype=compute_dtype if not use_quantization else None, # Set dtype if not quantizing
    device_map="auto", # Automatically handle device placement
    trust_remote_code=True,
)

print(f"Loading tokenizer for: {base_model_id}")
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # Set pad token if needed
tokenizer.padding_side = "right"

# --- Load LoRA Adapter and Merge ---
print(f"Loading LoRA adapter from: {adapter_path}")
# Load the LoRA adapter weights on top of the base model
# Note: device_map="auto" should handle placing the adapter correctly
model = PeftModel.from_pretrained(model, adapter_path)
print("LoRA adapter loaded.")

print("Merging LoRA adapter into the base model...")
# Merge the adapter weights into the base model weights
# This returns a standard transformers model object
model = model.merge_and_unload()
print("Adapter merged.")

# Set the model to evaluation mode
model.eval()

# --- Prepare Prompt using the Correct Chat Template ---
# *** CRITICAL: Use the exact same format as your training data ***
# Example Llama 3.2 Instruct format (VERIFY THIS STRUCTURE!)
# prompt_text = "Investigating the effects of manipulation on the gale shapley algorithm."

# Format the prompt using the chat template structure
# Ensure this matches the format_dataset_batch function from your training script
formatted_prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>

{prompt_text}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

""" # Note the trailing space and newline for the assistant's turn

print("\nFormatted Prompt:")
print(formatted_prompt)

# --- Tokenize Input ---
print("Tokenizing prompt...")
# Encode the formatted prompt into token IDs
# Ensure tensors are moved to the same device as the model
inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=False, truncation=True).to(device)

# --- Generate Output ---
print("Generating response...")
# Generate text using the model
# Adjust generation parameters as needed (max_new_tokens, temperature, top_p, etc.)
with torch.no_grad(): # Disable gradient calculations for inference
    outputs = model.generate(
        **inputs,
        max_new_tokens=2048,          # Maximum number of new tokens to generate
        do_sample=True,             # Whether to use sampling; set to False for deterministic output
        temperature=0.6,            # Controls randomness (lower = more deterministic)
        top_p=0.9,                  # Nucleus sampling probability
        eos_token_id=tokenizer.eos_token_id # Stop generation when EOS token is encountered
    )

# --- Decode and Print Output ---
# Decode the generated token IDs back into text
# Skip special tokens (like padding or EOS) in the output string
# We slice the output to only decode the newly generated tokens (after the input prompt)
generated_token_ids = outputs[0][inputs['input_ids'].shape[1]:]
response = tokenizer.decode(generated_token_ids, skip_special_tokens=True)

div = "="*80
print(f"\n{div}\nModel Response:")
display(Markdown(response))


# print(response)

print("\nInference complete.")



/usr/local/lib/python3.11/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/usr/local/lib/python3.11/dist-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
2025-04-22 15:26:01.868674: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-22 15:26:01.868734: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-22 15:26:01.869805: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:

[2025-04-22 15:26:03,615] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.6
 [WARNING]  using untested triton version (3.2.0), only 1.0.0 is known to be compatible


/usr/local/lib/python3.11/dist-packages/deepspeed/runtime/zero/linear.py:47: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @autocast_custom_fwd
/usr/local/lib/python3.11/dist-packages/deepspeed/runtime/zero/linear.py:66: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @autocast_custom_bwd


...

Loading LoRA adapter from: ./x_results_llama3_finetuned
LoRA adapter loaded.
Merging LoRA adapter into the base model...
Adapter merged.

Formatted Prompt:
<|begin_of_text|><|start_header_id|>user<|end_header_id|>

Investigating the effects of manipulation on the gale shapley algorithm.<|eot_id|><|start_header_id|>assistant<|end_header_id|>


Tokenizing prompt...


Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


Generating response...

Model Response:


The Gale-Shapley algorithm is a well-known algorithm used in stable marriage problem to match entities (e.g., men and women) in a way that maximizes the number of stable pairings. Manipulation in this context refers to finding or designing strategies that deviate from the algorithm’s output to achieve a different stable or suboptimal matching.

**Research Problem:**  
Investigate how manipulation affects the Gale-Shapley algorithm's stability, efficiency, and fairness.

---

### Approach

1. **Literature Review**  
   - Study existing theoretical results on the Gale-Shapley algorithm and known manipulation strategies.  
   - Review previous empirical studies on the robustness of the algorithm against manipulation.

2. **Theoretical Analysis**  
   - Formalize different types of manipulation (e.g., prefix manipulation, postfix manipulation, and simultaneous manipulation).  
   - Analyze the impact of each type on the algorithm's output.  
   - Derive conditions under which manipulation succeeds or fails.

3. **Simulation and Experimentation**  
   - Implement the Gale-Shapley algorithm and various manipulation strategies.  
   - Generate synthetic datasets reflecting realistic pairing scenarios (e.g., different numbers of entities, preferences).  
   - Run simulations to observe the outcomes and measure success rates of manipulation attempts.

4. **Case Studies**  
   - Analyze specific instances where manipulation succeeds or fails.  
   - Interpret results to understand the impact on stability and fairness.

5. **Comparison with Alternative Algorithms**  
   - Compare the Gale-Shapley algorithm’s robustness against manipulation with other matching algorithms (e.g., Hungarian algorithm, block design algorithms).

### Data Collection Plan

- Collect synthetic data from generated random preference lists and matching results.  
- Record manipulation attempts and outcomes.  
- Document stability metrics (number of stable pairings, number of manipulations successful).

### Data Analysis Plan

- Statistical analysis to compare success rates of manipulation across different manipulation types and dataset sizes.  
- Visualization of algorithm performance metrics over varying numbers of entities.  
- Theoretical analysis to derive bounds on manipulation success probability.

### Potential Challenges or Limitations

- Computational complexity in simulating large-scale scenarios.  
- Difficulty in generalizing results across diverse preference structures.  
- Defining and measuring "success" of manipulation in a formal way.

### Ethical Considerations

- Ensure fairness in the representation of preferences and entities.  
- Transparency in reporting manipulation results without bias.

---

This structured approach aims to comprehensively understand and quantify the effects of manipulation on the Gale-Shapley algorithm, contributing to both theoretical insights and practical improvements.


Inference complete.
